In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
import pandas as pd

###### VISÃO PROJEÇÃO TRANSACIONAL PRODUÇÃO

dataset1001 = pd.read_csv("../input/sep-dataset-1001/20161231_20200126_safras.csv",infer_datetime_format=True,
                       parse_dates = ['DATA'])

dataset = dataset1001.copy()
dataset.head()

daily_df = dataset[['DATA','FL_IDADE','TRN_TOTAL']][dataset['FL_ATIVO'] == 1]

df1 = daily_df[['DATA','TRN_TOTAL']][daily_df['FL_IDADE'].isin(['2+'])]
df2 = daily_df[['DATA','TRN_TOTAL']][daily_df['FL_IDADE'].isin(['1+'])]
df3 = daily_df[['DATA','TRN_TOTAL']][daily_df['FL_IDADE'].isin(['1-'])]
df4 = daily_df[['DATA','TRN_TOTAL']][daily_df['FL_IDADE'].isin(['0'])]

df = [df1, df2, df3, df4]
for var in range(0,4):
    df[var] = df[var].groupby('DATA')['TRN_TOTAL'].sum()
    df[var] = df[var].resample('D').apply(sum)
    df[var] = df[var].reset_index()
    df[var].columns = ['ds', 'y']

#################################################################################################################
#################################################################################################################

hldys1 = pd.DataFrame({
  'holiday': 'queda',
  'ds': pd.to_datetime([
                        '2017-02-28', '2017-04-14', '2017-04-16', '2017-04-21', '2017-05-01', '2017-06-15', 
                        '2017-09-07', '2017-10-12', '2017-11-02', '2017-11-15',
                        '2018-02-12', '2018-02-13', '2018-03-30', '2018-04-01', '2018-04-21', '2018-05-01', '2018-05-31',
                        '2018-09-07', '2018-10-12', '2018-11-02', '2018-11-15', '2019-03-04', '2019-03-05','2019-04-19',
                        '2019-04-21', '2019-05-01', '2019-06-20', '2019-09-07', '2019-10-12','2019-11-02','2019-11-15']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys2 = pd.DataFrame({
  'holiday': 'alta_sep',
  'ds': pd.to_datetime(['2017-12-20','2018-12-20','2019-12-20']),
  'lower_window': 0,
  'upper_window': 2,
})

hldys3 = pd.DataFrame({
  'holiday': 'especial',
  'ds': pd.to_datetime(['2017-12-25','2018-01-01','2018-12-25','2019-01-01','2019-12-25','2020-01-01','2020-12-25']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys4 = pd.DataFrame({
  'holiday': 'alta',
  'ds': pd.to_datetime(['2017-02-24','2017-03-01','2017-03-02', '2017-06-10', '2017-08-12', '2017-11-24', #'2017-05-13',
                        '2018-02-09','2018-02-14','2018-02-15', '2018-06-12', '2018-08-11', '2018-11-23',#'2018-05-12',
                        '2019-03-01','2019-03-06','2019-03-07','2019-05-11','2019-06-12','2019-08-10','2019-11-29']),
  'lower_window': 0,
  'upper_window': 0,
})

hldys = pd.concat((hldys1, hldys2, hldys3, hldys4))

prediction_size = 160
train_df = [df[0], df[1], df[2], df[3]]

i = 0
for var in range(0,4):
    train_df[var] = df[i].copy()
    i = i + 1
    train_df[var]['y'] = train_df[var]['y'][train_df[var]['y'] != 0]
    train_df[var]['y'] = np.log(train_df[var]['y'])

#################################################################################################################
#################################################################################################################
# 0 weekly seasonality prior_scale
# 1 monthly seasonality prior_scale
# 2 seasonality prior_scale
# 3 holidays seasonality prior_scale
# 4 changepoint_prior_scale
# 5 changepoint_range
# 6 fourier_order semana
# 7 fourier_order mes
# 8 fourier_order ano
    # 1a coluna: 1o dataframe
par = [[70,70,31,46,0.35,0.59,50,20,1], [70,70,31,46,0.35,0.59,50,20,1], 
       [70,70,31,46,0.35,0.59,50,20,1], [70,70,31,46,0.01,0.59,50,20,1]]
forecast = [0,0,0,0,0]
for i in range(0,4):
    m = Prophet(yearly_seasonality=False,weekly_seasonality=False,growth = 'linear',
                daily_seasonality=False, holidays = hldys, holidays_prior_scale = par[i][3],
               seasonality_prior_scale = par[i][2], seasonality_mode = 'additive',
               changepoint_prior_scale = par[i][4] , changepoint_range = par[i][5],
               ).add_seasonality(
                name='w',period=7,fourier_order = par[i][6], prior_scale = par[i][0]).add_seasonality(
                name='m',period=30.5,fourier_order = par[i][7], prior_scale = par[i][1]).add_seasonality(
                name='ano',period=365,fourier_order = par[i][8], prior_scale = par[i][8])
    m.fit(train_df[i])
    future = m.make_future_dataframe(periods=prediction_size)
    forecast[i] = m.predict(future)

#################################################################################################################
#################################################################################################################
a = [0,0,0,0]
for i in range(0,4):
    a[i] = forecast[i].set_index('ds')
    a[i] = a[i]['yhat']
    a[i] = np.exp(a[i])

forecast[3].head(208)
np.exp(10)
a[3]['2020-01-29':'2020-01-29'].sum()

pd.plotting.register_matplotlib_converters()
m.plot_components(forecast[3])
#################################################################################################################
#################################################################################################################

b = dataset.set_index('DATA')['2020-01-01':'2020-01-26']['TRN_TOTAL'].sum()
c = (a[0]['2020-01-27':'2020-01-31'].sum() + a[1]['2020-01-27':'2020-01-31'].sum()
        + a[2]['2020-01-27':'2020-01-31'].sum() + a[3]['2020-01-27':'2020-01-31'].sum())
est = b + c
print(b)
print(est)

#################################################################################################################
#################################################################################################################